# Data Scraping

In [ ]:
import pandas as pd
import numpy as np
import requests as r 

### Downloading and reading the files

In [ ]:
def download_files():
##################################################################################################
### This function downloads the datasets "données hospitalières" and "capacité d'hospitalisation"
### directly from the urls below. 
##################################################################################################
    url_donnees_hosp = "https://www.data.gouv.fr/fr/datasets/r/63352e38-d353-4b54-bfd1-f1b3ee1cabd7"
    url_capa_hosp = "https://drees.solidarites-sante.gouv.fr/IMG/xlsx/es_2019_fiche_02_capacite_.xlsx"
    
    donnees_hosp = r.get(url_donnees_hosp)
    donnes_capa = r.get(url_capa_hosp)
    
    with open('dh.csv', 'wb') as f:
        f.write(donnees_hosp.content)
    with open('dc.xlsx', 'wb') as f:
        f.write(donnes_capa.content)
        
    
def read_files():
####################################################################################################
#### This function read the 2 data files downloaded, and other datasets manually downloaded which
#### are : "nombre de médecins par départeement" and  "population par département"
##### df = données_hospitalières groupées = nombre de cas covid par jour par département
##### dfc = capacité des hopitaux par département (par 10 000 habitants)
##### dfs = nombre de médecins par département
##### dfp = population par département
####################################################################################################

    df = pd.read_csv("./dh.csv",sep=";")
    df = df.groupby(["jour","dep"]).sum()
    df = df.drop(["sexe","rad","dc"],axis=1)
    
    dfc = pd.read_excel("./dc.xlsx",sheet_name = "ES_2019_fiche 02_carte 2",skiprows=3,usecols="b:d",nrows=101)
    
    dfs = pd.read_excel("./staff.xls",skiprows=4,skipfooter=2,usecols=[0,1,4,5,7])
    
    dfp = pd.read_csv("./pop.csv",skiprows=0,usecols=[0,6],sep=";")

    return df, dfc, dfs, dfp

### Computing metrics and generating the final dataset

**The metrics we used in our dashboard are computed as follows:**

$\text{Hospitalization rate (global)} = \frac{\text{Number of hospitalized people}}{\text{Hospital capacity}}*100$

$\text{Hospitalization rate (intensive care)} = \frac{\text{Number of hospitalized people (intensive care only)}}{\text{Hospital capacity}}*100$

$\text{Medical staff saturation} = \frac{\text{Total number of medical staff}}{\text{Number of hospitalized people}}$

In [ ]:
def data_processing():
##########################################################################################################
#### This function cleans and processes the data downloaded and generates an excel file that contains only 
#### the attributes we want:
#### - Hospitalization rate (global)
#### - Hospitalization rate (intensive care only)
#### - Medical staff saturation
#########################################################################################################
    capa = read_files()[1].rename({"Nombre de places p10 000":"capa","Département":"dep"},axis=1)
    cas = read_files()[0]
    staff = read_files()[2].rename({"Unnamed: 0":"dep","Unnamed: 1":"dep_name","dont général.":"general",
                                   "dont spécial.":"spe","Infirmiers diplômés d’État":"infirmiers"},axis=1)
    pop = read_files()[3]
    pop["p_tot_10k"] = pop["PTOT"]/10000
    
    metrics = pd.merge(cas.reset_index(),capa,how="left",left_on="dep",right_on="dep")
    metrics = pd.merge(metrics,staff,how="left",left_on="dep",right_on="dep")
    
    metrics["tot_cas"] = metrics["hosp"] + metrics["rea"]
    metrics["tot_medecins"] = metrics["general"] + metrics["spe"]+metrics["infirmiers"]
    metrics = metrics.drop(["general","spe","infirmiers","dep_name"],axis=1)
    
    # metrics --> dep | capacité | totaux médecins (généraux + spe + infirmiers) | totaux cas (hosp + rea)

    metrics["nbre_medecins_par_cas"] = metrics["tot_medecins"]/metrics["tot_cas"]

    # metrics --> dep | capacité | totaux médecins | totaux cas | nb médecins par cas
    
    metrics = metrics.drop(["tot_cas"],axis=1) 

    metrics = pd.merge(metrics,pop,how="left",left_on="dep",right_on="CODDEP").drop(["CODDEP","PTOT"],axis=1)
    
    # metrics --> dep | 

    ##############
    metrics["taux_hosp"] = 100*metrics["hosp"]/(metrics["capa"]*metrics["p_tot_10k"])
    metrics["taux_rea"] = 100*metrics["rea"]/(metrics["capa"]*metrics["p_tot_10k"])
    ##############
    
    
    metrics[(metrics["dep"]=="75") & (metrics["jour"]=="2020-05-31")][["taux_rea","jour"]]

    metrics = metrics.fillna(0)

    metrics.loc[(metrics["nbre_medecins_par_cas"] == np.inf),"nbre_medecins_par_cas"] = metrics[(metrics["nbre_medecins_par_cas"] == np.inf)]["tot_medecins"]
    taux_hosp = pd.pivot_table(metrics,index="dep",columns="jour",values="taux_hosp")
    taux_rea = pd.pivot_table(metrics,index="dep",columns="jour",values="taux_rea")
    nbre_medecins_par_cas = pd.pivot_table(metrics,index="dep",columns="jour",values="nbre_medecins_par_cas")

    writer = pd.ExcelWriter("./metrics.xlsx", engine = 'xlsxwriter')
    taux_hosp.to_excel(writer, sheet_name = 'taux_hosp')
    taux_rea.to_excel(writer, sheet_name = 'taux_rea')
    nbre_medecins_par_cas.to_excel(writer, sheet_name = 'nbre_medecins_par_cas')

    writer.save()
    writer.close()

In [ ]:
if __name__== '__main__':
    download_files()
    data_processing()